# TWSE 每日股價更新爬蟲
檢視網頁原始碼及開發工具選項中觀察網頁。

In [34]:
import requests

res = requests.get('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20190516&type=ALLBUT0999&_=1520785530355')
line = res.text.split('\n')
line[0:10]

['"108年05月16日 大盤統計資訊"\r',
 '"指數","收盤指數","漲跌(+/-)","漲跌點數","漲跌百分比(%)",\r',
 '"寶島股價指數","11,984.80","-","105.66","-0.87",\r',
 '"發行量加權股價指數","10,474.61","-","86.10","-0.82",\r',
 '"臺灣公司治理100指數","5,941.06","-","40.80","-0.68",\r',
 '"臺灣50指數","7,881.89","-","64.70","-0.81",\r',
 '"臺灣中型100指數","7,630.00","-","103.56","-1.34",\r',
 '"臺灣資訊科技指數","10,274.74","-","93.76","-0.90",\r',
 '"臺灣發達指數","8,172.00","-","74.41","-0.90",\r',
 '"臺灣高股息指數","6,044.14","-","76.09","-1.24",\r']

### 檢查是否抓取到資料


In [35]:
#直接把csv檔存到df
import pandas as pd
from io import StringIO #讓字串變檔案

line = res.text.split('\n') #換行切
line[300]



'="01010T","京城樂富R1","33,000","9","329,780","9.99","10.00","9.99","10.00"," ","0.00","9.99","3","10.00","380","0.00",\r'

### for 迴圈處理資料

In [36]:
newdf = []

for li in line:
    if len(li.split('",')) > 10:  #", 切分隔符號 觀察上一個步驟，只要大於10的列
        newdf.append(li)
print(newdf[0])
print(newdf[1])

"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",
="0050","元大台灣50","17,099,974","7,751","1,355,088,953","79.50","79.60","79.00","79.05","-","0.70","79.00","1,522","79.05","1,591","0.00",


### dataframe製作

In [39]:
ncontent = "\n".join(newdf).replace('=','')
df = pd.read_csv(StringIO(ncontent))
df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
0,0050,元大台灣50,"17,099,974","7,751","1,355,088,953",79.50,79.60,79.00,79.05,-,0.70,79.00,"1,522",79.05,"1,591",0.00,NaN
1,0051,元大中型100,"27,031",19,"843,766",31.25,31.38,30.96,30.96,-,0.24,30.78,20,30.96,27,0.00,NaN
2,0052,富邦科技,"131,000",14,"6,923,450",53.00,53.00,52.45,52.45,-,0.60,52.35,1,52.80,1,0.00,NaN
3,0053,元大電子,"4,000",4,"135,910",33.98,34.08,33.83,33.83,-,0.32,33.76,2,33.77,30,0.00,NaN
4,0054,元大台商50,"5,000",3,"106,540",21.44,21.44,21.25,21.25,-,0.19,21.04,30,21.14,30,0.00,NaN


### 資料整理
1.字串->數字 

2.逗點,刪除

3.欄位unnamed:16刪除

In [52]:
df = df.astype(str) #都轉為字串

#def func(s):
#    return s.str.replace(','.'')

df = df.apply(lambda s: s.str.replace(',','')) #apply 對dataframe的每一條series都去做function的動作。

#df.index = df['證券代號']
#df = df.drop('證券代號', axis = 1)

df = df.apply(lambda s: pd.to_numeric(s, errors='coerce')) #errors='coerce' 轉換失敗的話就填成na

df = df[df.columns[df.isnull().sum() != len(df)]] #刪除na

df.head()


,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
0050,17099974,7751,1355088953,79.50,79.60,79.00,79.05,0.70,79.00,1522,79.05,1591,0.0
0051,27031,19,843766,31.25,31.38,30.96,30.96,0.24,30.78,20,30.96,27,0.0
0052,131000,14,6923450,53.00,53.00,52.45,52.45,0.60,52.35,1,52.80,1,0.0
0053,4000,4,135910,33.98,34.08,33.83,33.83,0.32,33.76,2,33.77,30,0.0
0054,5000,3,106540,21.44,21.44,21.25,21.25,0.19,21.04,30,21.14,30,0.0


In [53]:
df[df['收盤價']/df['開盤價'] > 1.05]

,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
4943,9916659,6848,1314657647,122.00,138.50,121.00,133.0,6.50,133.00,5,133.5,7,19.76
9157,3000,2,1720,0.56,0.60,0.56,0.6,0.00,0.56,10,0.6,7,0.00
9902,1844961,646,18222846,9.52,10.25,9.52,10.0,0.46,9.95,1,10.0,32,31.25
